In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential, load_model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

In [81]:
# df = pd.read_csv("Processed_data.csv")
# df.drop("Unnamed: 0",inplace=True,axis=1)
# y = df['final_score']
# df.drop('final_score',inplace=True,axis=1)
# X=df
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
# X_train.shape

df = pd.read_csv("training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)
df['domain1_score']=temp['final_score']

#y = df['domain1_score']
#df.drop('domain1_score',inplace=True,axis=1)
#X=df
y=temp['final_score']
temp.drop('final_score',inplace=True,axis=1)
temp.drop('spell_err_count',inplace=True,axis=1)

X=temp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [82]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [83]:
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)


In [94]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv[i])        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)
training_vectors.shape

C:\Users\Hasnat\AppData\Local\Temp\ipykernel_20428\2450526373.py:9: RuntimeWarning: invalid value encountered in divide
  vec = np.divide(vec,noOfWords)


(9083, 300)

In [95]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [96]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_13 (LSTM)              (None, 64)                93440     
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [97]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=100)

Epoch 1/100
142/142 [==============================] - 5s 13ms/step - loss: 7.4449 - mae: 2.1289
Epoch 2/100
142/142 [==============================] - 2s 13ms/step - loss: 5.2055 - mae: 1.7921
Epoch 3/100
142/142 [==============================] - 2s 13ms/step - loss: 4.8155 - mae: 1.7223
Epoch 4/100
142/142 [==============================] - 2s 13ms/step - loss: 4.6525 - mae: 1.6888
Epoch 5/100
142/142 [==============================] - 2s 13ms/step - loss: 4.5319 - mae: 1.6593
Epoch 6/100
142/142 [==============================] - 2s 13ms/step - loss: 4.5164 - mae: 1.6612
Epoch 7/100
142/142 [==============================] - 2s 13ms/step - loss: 4.3676 - mae: 1.6322
Epoch 8/100
142/142 [==============================] - 2s 14ms/step - loss: 4.2429 - mae: 1.6021
Epoch 9/100
142/142 [==============================] - 2s 14ms/step - loss: 4.2113 - mae: 1.5998
Epoch 10/100
142/142 [==============================] - 2s 14ms/step - loss: 4.1451 - mae: 1.5837
Epoch 11/100
142/142 [=======

In [98]:
lstm_model.save('lstm_model_ideal.h5')


In [99]:
lstm_model = load_model("lstm_model_combined.h5")
y_pred = lstm_model.predict(testing_vectors)
y_pred

122/122 [==============================] - 1s 4ms/step


array([[1.7246292],
       [0.9705539],
       [2.3294606],
       ...,
       [1.87343  ],
       [1.8651178],
       [4.3206987]], dtype=float32)

In [31]:
from gensim.models.keyedvectors import KeyedVectors

In [63]:
def sent2word(x):
    stop_words = set(stopwords.words('english')) 
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    #index2word_set = set(model.wv.index2word)
    index2word_set = set(model.index_to_key)
    
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model[i])        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs

def convertToVec(text):
    #print(text)
    content= text
    if len(content) > 20:
        num_features = 300
        model = KeyedVectors.load_word2vec_format("word2vecmodel.bin", binary=True)
        clean_test_essays = []
        clean_test_essays.append(sent2word(content))
        testDataVecs = getVecs(clean_test_essays, model, num_features )
        testDataVecs = np.array(testDataVecs)
        testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
        
        lstm_model = load_model("lstm_model.h5")
        preds = lstm_model.predict(testDataVecs)
        print(preds)
        #print('hi')
        return str(round(preds[0][0]))